# Introduction
#Customer segmentation is essential for modern businesses to understand their diverse customer base. By analyzing shopping mall customer data, we can uncover hidden patterns in purchasing behavior. This project applies machine learning to categorize customers into meaningful groups. These insights help businesses create targeted marketing strategies.

# Problem Statement
#Malls struggle to understand customer preferences due to diverse shopping behaviors. Traditional approaches fail to identify distinct customer segments for personalized marketing. This project solves this by using clustering algorithms to automatically group similar customers.

# Import all Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

In [ ]:
df=pd.read_csv("C:\\UMAR DATA\\Data Sciences\\Intership\\Developer Hub\\Phase 2\\Task 5\\Mall_Customers.csv")
df.head()

# Basic Data Understanding

In [ ]:
df=df.drop(['CustomerID'],axis='columns')
print(f'Shape of the Dataset:{df.shape}\n')
print(f'Null Values in Dataset:\n{df.isnull().sum()}\n')
print(f'Duplicated Values in Dataset:{df.duplicated().sum()}\n')
print(f'Drop Duplicated Values:{df.drop_duplicates(inplace=True)}\n')
print(f'Basic Information about Dataset:{df.info()}')

In [ ]:
df_cate=df.select_dtypes(include='object')
df_category=df_cate.columns
print(f'Total Numbers of Categorical variable is {len(df_category)}')
df_num=df.select_dtypes(include='number')
df_number=df_num.columns
print(f'Total Number of Numerical data is {len(df_number)}')

In [ ]:
print(f'Basic Summary of Numerical Features:\n{df.describe()}')

In [ ]:
print(f"Basic Summary of Categorical Features:\n{df.describe(include='object')}")

# Feature Engineering

In [ ]:
bins = [0, 18, 30, 45, 60, 100]
labels = ['Child', 'Young Adult', 'Adult', 'Middle Age', 'Senior']
df['age_category'] = pd.cut(df['Age'], bins=bins, labels=labels)
df.head()

In [ ]:
income_bins = [0, 40, 80, 120, 160]
income_labels = ['Low Income', 'Medium Income', 'High Income', 'Very High Income']
df['Income_Category'] = pd.cut(df['Annual Income (k$)'], bins=income_bins, labels=income_labels)
spending_bins = [0, 30, 60, 80, 100]
spending_labels = ['Low Spender', 'Moderate Spender', 'High Spender', 'Very High Spender']
df['Spending_Category'] = pd.cut(df['Spending Score (1-100)'], bins=spending_bins, labels=spending_labels)

# Exploratory Data Analysis

In [ ]:
count=df.age_category.value_counts()
index=np.argmax(count)
explode=[0.12 if i == index else 0.1 for i in range(len(count))]
plt.pie(count,labels=count.index,explode=explode,autopct='%1.1f%%',shadow=True)
plt.title("Distribution of Age Group")
plt.tight_layout()
plt.savefig("C:\\UMAR DATA\\Data Sciences\\Intership\\Developer Hub\\Phase 2\\Task 5\\Distribution of Age Group.png", 
            dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')

In [ ]:
count=df.Gender.value_counts()
index=np.argmax(count)
explode=[0.12 if i == index else 0 for i in range(len(count))]
plt.pie(count,labels=count.index,explode=explode,autopct='%1.1f%%',shadow=True)
plt.title("Distribution of Age Group")
plt.tight_layout()
plt.savefig("C:\\UMAR DATA\\Data Sciences\\Intership\\Developer Hub\\Phase 2\\Task 5\\Distribution of Gender.png", 
            dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')

In [ ]:
plt.figure(figsize=(4,5))
sns.histplot(data=df,x='Annual Income (k$)',kde=True)
plt.title("Distribution of Income")
plt.savefig("C:\\UMAR DATA\\Data Sciences\\Intership\\Developer Hub\\Phase 2\\Task 5\\Distribution of Income.png", 
            dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')

In [ ]:
plt.figure(figsize=(4,5))
sns.histplot(data=df,x='Spending Score (1-100)',kde=True)
plt.title("Distribution of Spending")
plt.savefig("C:\\UMAR DATA\\Data Sciences\\Intership\\Developer Hub\\Phase 2\\Task 5\\Distribution of Spending.png", 
            dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')

In [ ]:
sns.scatterplot(data=df,x='Annual Income (k$)',y='Spending Score (1-100)',
                hue='Gender',palette='Set1')
plt.title("Gender Spending")
plt.savefig("C:\\UMAR DATA\\Data Sciences\\Intership\\Developer Hub\\Phase 2\\Task 5\\Distribution of Gender_spending.png", 
            dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')

In [ ]:
male_df = df[df['Gender'] == 'Male']
female_df = df[df['Gender'] == 'Female']
fig, axes = plt.subplots(1, 2, figsize=(14, 6))
ax1 = sns.countplot(data=male_df, x='Income_Category', hue='Spending_Category', 
              ax=axes[0], palette='Blues')
axes[0].set_title('Male Customers')
axes[0].tick_params(axis='x', rotation=45)
axes[0].set_xlabel('Income Category')
axes[0].set_ylabel('Count')
for container in ax1.containers:
    ax1.bar_label(container, fontsize=9, padding=3)
ax2 = sns.countplot(data=female_df, x='Income_Category', hue='Spending_Category', 
              ax=axes[1], palette='Reds')
axes[1].set_title('Female Customers')
axes[1].tick_params(axis='x', rotation=45)
axes[1].set_xlabel('Income Category')
axes[1].set_ylabel('Count')
for container in ax2.containers:
    ax2.bar_label(container, fontsize=9, padding=3)
plt.tight_layout()
plt.savefig("C:\\UMAR DATA\\Data Sciences\\Intership\\Developer Hub\\Phase 2\\Task 5\\spending and Male Female.png", 
            dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')

# Clustering And PCA

In [ ]:
features = ['Annual Income (k$)', 'Spending Score (1-100)', 'Age']
X = df[features]
pipeline = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=2)),
    ('cluster', KMeans(n_clusters=5, random_state=42, n_init=10))
])
pipeline.fit(X)
clusters = pipeline.named_steps['cluster'].labels_
df['Cluster'] = clusters
X_scaled = pipeline.named_steps['scaler'].transform(X)
X_pca = pipeline.named_steps['pca'].transform(X_scaled)
df['PCA1'] = X_pca[:, 0]
df['PCA2'] = X_pca[:, 1]

# Visualize the clusters  

In [ ]:
plt.figure(figsize=(8, 6))
scatter=plt.scatter(df['PCA1'], df['PCA2'], c=df['Cluster'], cmap='Set1', alpha=0.7)
print(f"Cluster distribution:\n{df['Cluster'].value_counts().sort_index()}")
cbar=plt.colorbar(scatter)
cbar.set_label('Cluster', fontsize=12)
plt.grid(True, alpha=0.3)
plt.title('Customer Clusters Visualized with PCA')
plt.tight_layout()
plt.savefig("C:\\UMAR DATA\\Data Sciences\\Intership\\Developer Hub\\Phase 2\\Task 5\\Customer Clusters Visualized with PCA.png", 
            dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')